In [46]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd

In [41]:
# 옵션 설정 객체
options = Options()

# 브라이저 닫힘 방지
options.add_experimental_option('detach', True)

# 경고 제거
options.add_experimental_option('excludeSwitches', ['enable-logging'])

In [42]:
# 드라이버 클래스 선언
driver = webdriver.Chrome(ChromeDriverManager().install(),
                         options=options)

C:\Users\Jiseok\AppData\Local\Temp\ipykernel_17340\3694344865.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),


In [43]:
# 네이버쇼핑 카테고리 url 선언
url = 'https://search.shopping.naver.com/search/category/100002452?catId=50001079&origQuery&pagingIndex=1&pagingSize=40&productSet=total&query&sort=rel&timestamp=&viewType=list'

# url 접속
driver.get(url)

# 전체 방문해야하는 상세 카테고리 개수
num_visit = driver.find_element('xpath', '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/h4/a/span[4]').text
num_visit = int(num_visit)

# 방문해야하는 상세 카테고리 목록 수집
visit_list = [f'//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/ul/li[{i}]/a/span' for i in range(1, num_visit+1)]

In [44]:
visit_list

['//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/ul/li[1]/a/span',
 '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/ul/li[2]/a/span',
 '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/ul/li[3]/a/span',
 '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/ul/li[4]/a/span',
 '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/ul/li[5]/a/span',
 '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/ul/li[6]/a/span',
 '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/ul/li[7]/a/span',
 '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/ul/li[8]/a/span']

In [52]:
df_list = [] # 데이터프레임을 담을 data

In [53]:
# 카테고리 목록 방문
for category in visit_list:
    driver.find_element('xpath', category).click()
    time.sleep(2)
    
    # 현재 카테고리 정보 수집
    cat1 = driver.find_element('xpath', '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/h4/a/span[1]').text
    cat2 = driver.find_element('xpath', '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/h4/a/span[2]').text
    cat3 = driver.find_element('xpath', '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/h4/a/span[3]').text
    cat4 = driver.find_element('xpath', '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/h4/a/span[4]').text
    
    # 카테고리 고유번호 확인
    current_url = driver.current_url
    cat_code = cat_code = re.findall('/[0-9]{9}', url)[0][1:]
    cat_id = cat_id = re.findall('=[0-9]{8}', url)[0][1:]
    
    # 인덱스 순차 방문
    for i in range(1, 2):
        target_url = f'https://search.shopping.naver.com/search/category/{cat_code}?catId={cat_id}&origQuery&pagingIndex={i}&pagingSize=80&productSet=total&query&sort=rel&timestamp=&viewType=list'
        driver.get(target_url)
        
        # 스크롤 끝까지 내리기
        while True:
            bh = driver.execute_script('return document.body.scrollHeight')

            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(1)

            ah = driver.execute_script('return document.body.scrollHeight')

            if ah == bh:
                break

            bh = ah

        # 크롤링 시작
        infos = driver.find_elements(By.CSS_SELECTOR, '.basicList_info_area__TWvzp')
        for info in infos:
            title = info.find_element(By.CSS_SELECTOR, '.basicList_title__VfX3c > a').text
            price = info.find_element(By.CSS_SELECTOR, 'div.basicList_info_area__TWvzp > div.basicList_price_area__K7DDT > strong > span > span.price_num__S2p_v').text
            data = [title, price, cat1, cat2, cat3, cat4]
            df_list.append(data)

In [54]:
df = pd.DataFrame(columns=['title', 'price', 'cat1', 'cat2', 'cat3', 'cat4'], data=df_list)

In [55]:
df_list

[['종근당건강 올앳미 V스파클링 저칼로리음료 24병', '22,900원', '식품', '생수/음료', '청량/탄산음료', '콜라'],
 ['종근당건강 올앳미 V스파클링 저칼로리음료 12병', '14,900원', '식품', '생수/음료', '청량/탄산음료', '콜라'],
 ['[24병]1am 스파클링 제로칼로리 제로탄산', '22,500원', '식품', '생수/음료', '청량/탄산음료', '콜라'],
 ['코카콜라 제로 슈가 콜라 190 355 500 ml 업소용 소매용 모음',
  '18,990원',
  '식품',
  '생수/음료',
  '청량/탄산음료',
  '콜라'],
 ['롯데칠성음료 펩시제로슈거 라임향 355ml', '10원', '식품', '생수/음료', '청량/탄산음료', '콜라'],
 ['종근당건강 올앳미 V스파클링 저칼로리음료 24병', '22,900원', '식품', '생수/음료', '청량/탄산음료', '사이다'],
 ['종근당건강 올앳미 V스파클링 저칼로리음료 12병', '14,900원', '식품', '생수/음료', '청량/탄산음료', '사이다'],
 ['[24병]1am 스파클링 제로칼로리 제로탄산', '22,500원', '식품', '생수/음료', '청량/탄산음료', '사이다'],
 ['코카콜라 제로 슈가 콜라 190 355 500 ml 업소용 소매용 모음',
  '18,990원',
  '식품',
  '생수/음료',
  '청량/탄산음료',
  '사이다'],
 ['롯데칠성음료 펩시제로슈거 라임향 355ml', '10원', '식품', '생수/음료', '청량/탄산음료', '사이다'],
 ['종근당건강 올앳미 V스파클링 저칼로리음료 24병', '22,900원', '식품', '생수/음료', '청량/탄산음료', '이온음료'],
 ['종근당건강 올앳미 V스파클링 저칼로리음료 12병', '14,900원', '식품', '생수/음료', '청량/탄산음료', '이온음료'],
 ['[24병]1am 스파클링 제로칼로리 제로탄산', '22,500원', '식품', '생수/

In [56]:
df

,title,price,cat1,cat2,cat3,cat4
0,종근당건강 올앳미 V스파클링 저칼로리음료 24병,"22,900원",식품,생수/음료,청량/탄산음료,콜라
1,종근당건강 올앳미 V스파클링 저칼로리음료 12병,"14,900원",식품,생수/음료,청량/탄산음료,콜라
2,[24병]1am 스파클링 제로칼로리 제로탄산,"22,500원",식품,생수/음료,청량/탄산음료,콜라
3,코카콜라 제로 슈가 콜라 190 355 500 ml 업소용 소매용 모음,"18,990원",식품,생수/음료,청량/탄산음료,콜라
4,롯데칠성음료 펩시제로슈거 라임향 355ml,10원,식품,생수/음료,청량/탄산음료,콜라
5,종근당건강 올앳미 V스파클링 저칼로리음료 24병,"22,900원",식품,생수/음료,청량/탄산음료,사이다
6,종근당건강 올앳미 V스파클링 저칼로리음료 12병,"14,900원",식품,생수/음료,청량/탄산음료,사이다
7,[24병]1am 스파클링 제로칼로리 제로탄산,"22,500원",식품,생수/음료,청량/탄산음료,사이다
8,코카콜라 제로 슈가 콜라 190 355 500 ml 업소용 소매용 모음,"18,990원",식품,생수/음료,청량/탄산음료,사이다
9,롯데칠성음료 펩시제로슈거 라임향 355ml,10원,식품,생수/음료,청량/탄산음료,사이다
